In [ ]:
import pandas as pd
import os
from datetime import datetime

# Replace this with your actual filename
file_path = r"data\Final_Processed_Contacts(Matin_Final).xlsx"

# Load Excel file
df = pd.read_excel(file_path, dtype=str)

# Ensure comparison_logs folder exists
os.makedirs("single_dedupe_logs", exist_ok=True)

# Ensure comparison_logs folder exists
os.makedirs("single_dedupe_output", exist_ok=True)

# Detect fully duplicated rows
duplicates = df[df.duplicated()]
print(f"Total full-row duplicates: {len(duplicates)}")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")


df_cleaned = df.drop_duplicates(subset=["URL"], keep="first")
print(f"Total rows: {len(df)}")
print(f"Total rows after cleaning: {len(df_cleaned)}")

# Save the cleaned result
with pd.ExcelWriter(f"single_dedupe_output/cleaned_data_{timestamp}.xlsx", engine="openpyxl") as writer:
    # Write DataFrame to Excel (default sheet: Sheet1)
    df_cleaned.to_excel(writer, index=False, sheet_name="Sheet1")

    # Access the worksheet
    worksheet = writer.sheets["Sheet1"]

    # 1. Force 'Number' column to be treated as text in Excel
    if "Number" in df_cleaned.columns:
        number_col_idx = df_cleaned.columns.get_loc("Number")  # e.g. 3
        number_col_letter = chr(ord('A') + number_col_idx)     # Convert to Excel column letter, e.g. "D"

        # Format the 'Number' column as Text (Excel format "@")
        for cell in worksheet[f"{number_col_letter}2:{number_col_letter}{len(df_cleaned)+1}"]:
            for c in cell:
                c.number_format = "@"

    # 2. Auto-adjust column widths based on content length
    for column_cells in worksheet.columns:
        # Find max content length in the column (header + data)
        max_length = max(
            len(str(cell.value)) if cell.value is not None else 0
            for cell in column_cells
        )
        adjusted_width = max_length + 2  # Padding for readability
        col_letter = column_cells[0].column_letter
        worksheet.column_dimensions[col_letter].width = adjusted_width

print(f"Final cleaned data saved to single_dedupe_output/cleaned_data_{timestamp}.xlsx")

# Save the rows that were removed during cleaning for audit purposes
removed_rows = df.loc[~df.index.isin(df_cleaned.index)]
removed_rows.to_excel(f"single_dedupe_logs/actually_removed_{timestamp}.xlsx", index=False)

Total full-row duplicates: 0
Total rows: 1234
Total rows after cleaning: 1234
Final cleaned data saved to single_dedupe_output/cleaned_data_20250527_094832.xlsx


OSError: Cannot save file into a non-existent directory: 'single_dedupe_logs\actually_removed_data'